# Lab - 9

Name: Purusharth Malik

Registration No.: 2348542

### 1. Implement the Edit Distance

In [1]:
def editDistance(str1, str2, m, n):
    if m == 0:
        return n
    if n == 0:
        return m

    if str1[m-1] == str2[n-1]:
        return editDistance(str1, str2, m-1, n-1)
    
    return 1 + min(editDistance(str1, str2, m, n-1),    # Insert
                   editDistance(str1, str2, m-1, n),    # Remove
                   editDistance(str1, str2, m-1, n-1)    # Replace
                   )


In [3]:
str1=input("Enter First String:")
str2=input("Enter Second String:")
print(editDistance(str1, str2, len(str1), len(str2)))

Enter First String:sunday
Enter Second String:monday
2


### 2. Implement the Story or Poem Creator based on the keywords using NLG.

#### Importing Libraries

In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.utils as ku 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

#### Loading the data

In [8]:
data = open('poem.txt', encoding="utf8").read()
corpus = data.lower().split("\n")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)
 
print("Total Words:", total_words)

Total Words: 4567


#### Creating Text Embeddings

In [5]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = ku.to_categorical(label, num_classes=total_words+1)

#### Bi-directional LSTM Model

In [6]:
model = Sequential()
model.add(Embedding(total_words+1, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words+1/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words+1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 100)           380800    
                                                                 
 bidirectional (Bidirection  (None, 15, 300)           301200    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 15, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 3807)              384507    
                                                                 
 dense_1 (Dense)             (None, 3808)              14500864  
                                                        

In [7]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
510/510 [==============================] - 25s 47ms/step - loss: 6.9543 - accuracy: 0.0605
Epoch 2/100
510/510 [==============================] - 28s 55ms/step - loss: 6.5300 - accuracy: 0.0630
Epoch 3/100
510/510 [==============================] - 28s 56ms/step - loss: 6.3475 - accuracy: 0.0652
Epoch 4/100
510/510 [==============================] - 30s 59ms/step - loss: 6.2173 - accuracy: 0.0727
Epoch 5/100
510/510 [==============================] - 31s 61ms/step - loss: 6.1001 - accuracy: 0.0800
Epoch 6/100
510/510 [==============================] - 30s 60ms/step - loss: 5.9669 - accuracy: 0.0893
Epoch 7/100
510/510 [==============================] - 32s 63ms/step - loss: 5.8469 - accuracy: 0.0991
Epoch 8/100
510/510 [==============================] - 32s 63ms/step - loss: 5.7302 - accuracy: 0.1071
Epoch 9/100
510/510 [==============================] - 32s 63ms/step - loss: 5.6269 - accuracy: 0.1104
Epoch 10/100
510/510 [==============================] - 31s 61ms/step - l

#### Poem Generation

In [11]:
seed_text = input("Enter first few words")
next_words = 50
ouptut_text = ""
 
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1,
      padding='pre')
    predicted = np.argmax(model.predict(token_list, 
                                        verbose=0), axis=-1)
    output_word = ""
     
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
             
    seed_text += " " + output_word
     
print(seed_text)

valley the merry ladies in london were spreading as she stepped away from me and she moved through the fair mountain stout maid thou say god as we say a hummin sweeps it in that pretty fair maid i did salute her every valley fair black i used to frequent do0



# End